**Название проекта**:  
Оператор мобильной связи «Мегалайн»   
_____
**Описание исследования**:  
Многие клиенты Мегалайна пользуются архивными тарифами. Мегалайн хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».
Имеются данные о поведении клиентов, которые уже перешли на эти тарифы. Нужно построить модель для задачи классификации, которая выберет подходящий тариф. 
_____
**Цель исследования**:  
Построение модели классификации, способной отличить клиентов, переходящих на тарифы "Ультра", от клиентов, остающихся на тарифах "Смарт"
_____
**План исследования.**  
[Шаг 1. Загрузка файла с данными и изучение общей информации](#step1)  
[Шаг 2. Разделение исходных данных на обучающую, валидационную и тестовую выборки](#step2)  
[Шаг 3. Исследование качества разных моделей](#step3)  
[Шаг 4. Проверка качества модели на тестовой выборке](#step4)  
[Шаг 5. Общие результаты и выводы](#step6)  

_____
**Исходные данные.**
Исходный файл <a HREF='user_behavior.csv'>user_behavior.csv</a>, содержащий информацию о поведении одного пользователей за месяц.  
поля файла:
- сalls — количество звонков
- minutes — суммарная длительность звонков в минутах
- messages — количество sms-сообщений
- mb_used — израсходованный интернет-трафик в Мб
- is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).


<a id='step1'></a>
## Шаг 1. Загрузка файла с данными и изучение общей информации

In [185]:
# импортируем библиотеки

import pandas as pd
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier

In [186]:
try:
    df = pd.read_csv('users_behavior.csv')
    print('local load successful')
except:
    print('local load failed, loading from the inernet')
    df = pd.read_csv('https://code.s3.yandex.net/datasets/users_behavior.csv')

local load successful


In [187]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [188]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [189]:
df.is_ultra.unique()

array([0, 1], dtype=int64)

In [190]:
df.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


Данные достаточно распределены, средние к медианам достаточно близки  
целевая переменная `is_ultra` представляет собой значения 0 или 1  
целевая переменная не вполне сбалансирована (около 30% значений 1 и соответственно 70% значений 0)
необходимо выбрать модели классификации, подобрать параметры и обучить наиболее подходящую

<a id='step2'></a>
## Шаг 2. Разделение исходных данных на обучающую, валидационную и тестовую выборки

In [191]:
features = df.drop('is_ultra', axis='columns')
target = df['is_ultra']

In [192]:
# сначала отделяем 10% записей для тестовой выборки, на которой будем сравнивать выбранные модели

features_train, x_test, target_train, y_test = tts(features, target, test_size=0.1, random_state=54321)

In [193]:
# от оставшихся 90% 75% отправляем в обучающую выборку, 25% - в валидационную

x_train, x_valid, y_train, y_valid = tts(features_train, target_train, test_size=0.25, random_state=54321)

In [194]:
# проверяем размеры выборок

display(x_train.shape)
display(y_train.shape)
display(x_valid.shape)
display(y_valid.shape)
display(x_test.shape)
display(y_test.shape)

(2169, 4)

(2169,)

(723, 4)

(723,)

(322, 4)

(322,)

<a id='step3'></a>
## Шаг 3. Исследование качества разных моделей

In [195]:
# пишем процедуру, которая обучает инициализированную модель, сравнивает метрики на валидационной выборке и возвращает
# новую метрику и модель если метрика выше
# старую метрику и модель, если улучшения нет
def process_model(model, hyper, best_metric, best_model, best_hyper):
    model.fit(x_train, y_train)
    y_pred = model.predict(x_valid)
    metric = accuracy_score(y_valid, y_pred)
    if metric < best_metric:
        return best_metric, best_model, best_hyper
    else:
        return metric, model, hyper

In [196]:
# небольшая процедура для запуска выбора лучших параметров
def start():
    global best_metric
    global best_model
    global best_hyper
    best_metric = -1.0
    best_model = None
    best_hyper = ''
    return

In [197]:
# небольшая процедура для завершения выбора модели с лучшими параметрами:
# выводим на экран хначение метрики, добавляем лучшую модель в список выбранных моделей
def finish():
    global models
    models.append(best_model)
    print(f'accuracy: {best_metric}\nmodel type: {type(best_model)}\nhyper parameters: {best_hyper}')

In [198]:
# создаем список для моделей с выбранными наилучшими параметрами
models = []

In [199]:
# подбираем параметры для DecisionTreeClassifier
start()
for i in range(1, 10):
    model = DecisionTreeClassifier(max_depth=i, random_state=54321)
    hyper = f'max_depth={i}'
    best_metric, best_model, best_hyper = process_model(model, hyper, best_metric, best_model, best_hyper)
finish()

accuracy: 0.7980636237897649
model type: <class 'sklearn.tree._classes.DecisionTreeClassifier'>
hyper parameters: max_depth=4


In [200]:
# подбираем параметры для RandomForestClassifier
start()
for i in range(1, 10):
    for j in range(1, 40, 4):
        model = RandomForestClassifier(max_depth=i, n_estimators=j, random_state=54321)
        hyper = f'max_depth={i}, n_estimators={j}'
        best_metric, best_model, best_hyper = process_model(model, hyper, best_metric, best_model, best_hyper)
finish()

accuracy: 0.8201936376210235
model type: <class 'sklearn.ensemble._forest.RandomForestClassifier'>
hyper parameters: max_depth=8, n_estimators=21


In [201]:
# подбираем параметры для AdaBoostClassifier
start()
for j in range(1, 40):
    model = AdaBoostClassifier(n_estimators=j, random_state=54321)
    hyper = f'n_estimators={j}'
    best_metric, best_model, best_hyper = process_model(model, hyper, best_metric, best_model, best_hyper)
finish()

accuracy: 0.8077455048409405
model type: <class 'sklearn.ensemble._weight_boosting.AdaBoostClassifier'>
hyper parameters: n_estimators=32


In [202]:
# у логистрической регрессии нет параметров
start()
model = LogisticRegression()
hyper = ''
best_metric, best_model, best_hyper = process_model(model, hyper, best_metric, best_model, best_hyper)
finish()

accuracy: 0.7095435684647303
model type: <class 'sklearn.linear_model._logistic.LogisticRegression'>
hyper parameters: 


In [203]:
# у наивного байеса нет параметров
start()
model = GaussianNB()
hyper = ''
best_metric, best_model, best_hyper = process_model(model, hyper, best_metric, best_model, best_hyper)
finish()

accuracy: 0.7952973720608575
model type: <class 'sklearn.naive_bayes.GaussianNB'>
hyper parameters: 


In [204]:
# подбираем параметры для KNeighborsClassifier
start()
for i in range(1, 9):
    for j in ['auto', 'ball_tree', 'kd_tree', 'brute']:
        model = KNeighborsClassifier(n_neighbors=i, algorithm=j)
        hyper = f'n_neighbors={i}, algorithm={j}'
        best_metric, best_model, best_hyper = process_model(model, hyper, best_metric, best_model, best_hyper)
finish()

accuracy: 0.7676348547717843
model type: <class 'sklearn.neighbors._classification.KNeighborsClassifier'>
hyper parameters: n_neighbors=8, algorithm=brute


|модель|accuracy|подобранные параметры|
|-|-|-|
|DecisionTreeClassifier|79.81%|max_depth=4|
|RandomForestClassifier|82.02%|max_depth=8, n_estimators=21|
|AdaBoostClassifier|80.77%|n_estimators=32|
|LogisticRegression|70.95%|n/a|
|GaussianNB|79.53%|n/a|
|KNeighborsClassifier|76.76%|n_neighbors=8, algorithm=brute|


<a id='step4'></a>
## Шаг 4. Проверка качества модели на тестовой выборке

In [205]:
for model in models:
    y_pred = model.predict(x_test)
    a = accuracy_score(y_test, y_pred)
    print(f'{type(model)}\t\t\t{a}')

<class 'sklearn.tree._classes.DecisionTreeClassifier'>			0.7608695652173914
<class 'sklearn.ensemble._forest.RandomForestClassifier'>			0.7919254658385093
<class 'sklearn.ensemble._weight_boosting.AdaBoostClassifier'>			0.7701863354037267
<class 'sklearn.linear_model._logistic.LogisticRegression'>			0.6770186335403726
<class 'sklearn.naive_bayes.GaussianNB'>			0.7795031055900621
<class 'sklearn.neighbors._classification.KNeighborsClassifier'>			0.7484472049689441


на тестовой выборке качество упало, но лидирующая модель осталась лидирующей:  
RandomForestClassifier с качеством почти 80%

<a id='step6'></a>
## Шаг 5. Общие результаты и выводы

Были проанализированы данные по поведению клиентов, выбран ряд моделей-кандидатов для построения модели предсказания перехода клиента на тариф "Ультра".  
В качестве метрики для оценки качества используется accuracy - доля предсказаний, совпавших с фактом.  
Ниже сравнение результатов работы моделей-кандитатов на валидационной и тестовых выборках:  
|модель|accuracy на валидационной выборке|accuracy на тестовой выборке|подобранные параметры|
|-|-|-|-|
|DecisionTreeClassifier|79.81%|76.09%|max_depth=4|
|RandomForestClassifier|82.02%|79.19%|max_depth=8, n_estimators=21|
|AdaBoostClassifier|80.77%|77.02%|n_estimators=32|
|LogisticRegression|70.95%|67.70%|n/a|
|GaussianNB|79.53%|77.95%|n/a|
|KNeighborsClassifier|76.76%|74.84%|n_neighbors=8, algorithm=brute|

Для предсказания выбираем модель RandomForestClassifier с гипер-параметрами max_depth=8, n_estimators=21.